In [1]:
# ============================================================
# TASK 2 — TRAFFIC SIGN RECOGNITION (AUTO-PATH + ZERO ERROR)
# ============================================================

# -------------------- STEP 1: UPLOAD + EXTRACT DATASET ZIP --------------------
import os
from google.colab import files
import zipfile

print("📢 Upload your traffic_signs.zip (with train/ & val/ folders)...")
uploaded = files.upload()

zip_name = list(uploaded.keys())[0]
print("📦 Uploaded:", zip_name)

# Extract ZIP
with zipfile.ZipFile(zip_name, "r") as z:
    z.extractall("/content/traffic_dataset")

print("✅ ZIP Extracted! Searching folders...")

# -------------------- STEP 2: AUTO-DETECT train/val PATHS --------------------
root_path = "/content/traffic_dataset"

train_path = None
val_path = None

for root, dirs, files in os.walk(root_path):
    if "train" in dirs and "val" in dirs:
        train_path = os.path.join(root, "train")
        val_path = os.path.join(root, "val")
        break

print("📁 Train Path:", train_path)
print("📁 Val Path:", val_path)

if train_path is None or val_path is None:
    raise Exception("❌ train/ or val/ folder not found! ZIP ko dubara check karo.")
else:
    print("✅ Correct paths detected! Moving to next step.")

# -------------------- STEP 3: DATA PREPROCESSING --------------------
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1
)

val_gen = ImageDataGenerator(rescale=1/255.)

train_data = train_gen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_gen.flow_from_directory(
    val_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

NUM_CLASSES = train_data.num_classes
print("🔢 Total Classes Found:", NUM_CLASSES)

# -------------------- STEP 4: BUILD CNN MODEL (EfficientNetB0) --------------------
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base.trainable = False  # freeze base

x = GlobalAveragePooling2D()(base.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(base.input, output)

model.compile(
    optimizer=Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print("✅ Model Ready!")
model.summary()

# -------------------- STEP 5: TRAIN THE MODEL --------------------
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

print("🎉 Training Completed!")

# -------------------- STEP 6: SAVE MODEL --------------------
model.save("/content/traffic_sign_model.h5")
print("💾 Model saved as: traffic_sign_model.h5")


📢 Upload your traffic_signs.zip (with train/ & val/ folders)...


Saving dataset.zip.zip to dataset.zip.zip
📦 Uploaded: dataset.zip.zip
✅ ZIP Extracted! Searching folders...
📁 Train Path: /content/traffic_dataset/dataset.zip/train
📁 Val Path: /content/traffic_dataset/dataset.zip/val
✅ Correct paths detected! Moving to next step.
Found 2 images belonging to 2 classes.
Found 2 images belonging to 2 classes.
🔢 Total Classes Found: 2
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
✅ Model Ready!


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,378,021 (16.70 MB)

...

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 1.0000 - loss: 0.5605 - val_accuracy: 0.5000 - val_loss: 0.7345
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - accuracy: 0.5000 - loss: 0.6494 - val_accuracy: 0.5000 - val_loss: 0.6950
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 804ms/step - accuracy: 0.5000 - loss: 0.7105 - val_accuracy: 0.5000 - val_loss: 0.7293
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step - accuracy: 0.5000 - loss: 0.8539 - val_accuracy: 0.5000 - val_loss: 0.7618
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 684ms/step - accuracy: 0.5000 - loss: 0.7270 - val_accuracy: 0.5000 - val_loss: 0.7412
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step - accuracy: 0.0000e+00 - loss: 1.1760 - val_accuracy: 0.5000 - val_loss: 0.7110
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - accuracy: 0.5000 - loss: 0.8767 - val_accuracy: 0.5000 - val_loss: 0.7027
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step - accuracy: 0.5000 - loss: 0.7447 - val_accuracy: 0.5000 - val_loss: 0.7020


🎉 Training Completed!
💾 Model saved as: traffic_sign_model.h5
